# 제5장 Factorization Machines(FM)

FM은 사용자와 아이템의 다양한 특성을 모델화함으로써 이러한 예측의 성능을 높이려는 방볍  
앞에서 설명했던 Matrix Factorization(MF)은 사용자의 취향과 아이템의 특성을 나타내는 특성값(feature)을 k개로 요약해서 추출하고, 이를 이용해 각 사용자의 각 아이템의 선호도를 예측하는 방식  
그런데 사용자의 취향과 아이템의 특성뿐 아니라 예측에 도움을 줄 수 있는 다른 변수가 존재할 수 있다.  
ex) 사용자 : 나이,성별,지역,인구 통계변수 / 아이템 : 감독,출연배우,장르  
이러한 다양한 변수를 종합해서 요인화(factorization)해 주는 방법이 FM  

## 5.1 FM의 표준식

FM의 기본 아이디어는 모든 변수와 그 변수들 간의 상호작용(interaction)을 고려해서 평점을 예측하는 것  
$\hat{y}(\mathbf{x}) = w_0 + \sum_{i=1}^{n} w_i x_i + \sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \mathbf{v}_i, \mathbf{v}_j \rangle x_i x_j$  
- $\hat{y}(\mathbf{x})$ : 예측값
- $w_0$ : 전역 bias (global bias)
- $w_i$ : 특성 $i$의 1차 가중치
- $x_i$ :  특성 $i$의 입력값
- $\mathbf{v}_i \in \mathbb{R}^k$ : 특성 $i$에 대한 k-차원 잠재 벡터 (latent vector)
- $\langle v_i,v_j \rangle$ : 잠재 벡터간의 내적
- n : 입력 변수의 수
- k : 잠재요인의 수  

<br>

여기서 v는 MF의 P 혹은 Q행렬처럼 각 변수를 k개의 특성값(feature value)으로 표현하는 잠재요인행렬로 생각하면 된다. 즉 변수가 n개이고 특성값이 K개이므로 nxk 행렬이다.  
하나의 변수인 $x_i$에 대해서 하나의 편향값($w_i$)과 k개의 특성값(features)을 갖게 된다.  
만일 x가 각 사용자와 아이템 각각을 나타내는 One-hot Encoding이고 사용자와 아이템 외에 다른 변수가 사용되지 않는다면 위 식은 결국 MF의 식과 같아진다.  
즉 **FM은 MF에 다른 변수를 추가할 수 있도록 좀 더 일반화한 모델**이다

## 5.2 FM 식의 변형

앞의 식은 변수 x의 모든 가능한 두 개의 조합으로 구성되어 있기 때문에 변수의 수가 늘어남에 따라 계산의 복잡도가 기하급수적으로 커진다.  
앞의 식의 경우 변수의 수 n에 대한 복잡도를 나타내는 함수는 $O(kn^2)$이 된다  
![img.png](image/img.png)